In [1]:
# useing this variable for saving models and summaries
exp_name = "WCNNRNN_Wmultitask_32_opp"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../../src')

In [3]:
import models.WCNNRNN_model_2 as WCNNRNN_model
import training
import evaluation.nn_eval_multitask as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../../data/opp8.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((691, 1024, 77), (691, 1024, 19))

In [7]:
test_data.shape, test_gt.shape

((119, 1024, 77), (119, 1024, 19))

## Model

In [8]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

wavelet_levels = 6    #number of wavelet layer to have in the network
convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [4, 4, 2]
weights_shape = [[9, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
wavelet_weights_shape = [[5, n_variables, n_variables]]    #high_pass and low_pass dimensions
activation = tf.nn.relu

lstm_n_hidden = 128
n_classes = 19

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)

pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)

variables_weights = np.ones((n_classes,))

In [9]:
pos_weight.shape

(19,)

In [10]:
pos_weight

array([  0.28595522,  42.44204322,  42.96023857,  45.06666667,
        44.49794239,  34.32268371,  35.42833608,  50.54312354,
        54.97974684,  62.17714286,  69.87179487,  66.41463415,
        71.02605863,  57.96533333,  57.4973545 ,  39.35036496,
        12.09177028,  47.81236203,   4.77487595])

In [12]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [13]:
summary_path = '../../summary/' + exp_name 
checkpoint_path = '../../model/' + exp_name + '/' + exp_name 

In [14]:
model = WCNNRNN_model.Model(n_variables = n_variables,
                            learning_rate = learning_rate,
                            n_classes = n_classes)

In [15]:
model.build_model(pos_weight = pos_weight, 
                 learning_rate = learning_rate,
                 wavelet_levels = wavelet_levels,    
                 convolution_levels = convolution_levels,    
                 pooling_stride = pooling_stride,     
                 weights_shape = weights_shape,
                 bias_shape = bias_shape,
                 wavelet_weights_shape = wavelet_weights_shape,    
                 activation = activation,
                 lstm_n_hidden = lstm_n_hidden,
                 trainable_wavelet = True,
                 trainable_conv = True,
                 trainable_last = True,
                 variables_weights = variables_weights
)


Tensor("Squeeze_21:0", shape=(?, ?, 19), dtype=float32) Tensor("Reshape_7:0", shape=(?, ?, 19), dtype=float32)


In [16]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'multitask')

In [17]:
trainer.train(max_iter = 10000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Training: 0
ACTIVITY RECOGNITION
AUC = 0.066 vs. 0.068
F1 = 0.102 vs. 0.099
Accuracy = 0.428 vs. 0.056
CHANGE DETECTION
AUC = 0.154 vs. 0.162
F1 = 0.290 vs. 0.280
Precision = 0.170 vs. 0.163
Recall = 0.987 vs. 0.973
Validation: 0
ACTIVITY RECOGNITION
AUC = 0.064 vs. 0.062
F1 = 0.096 vs. 0.087
Accuracy = 0.507 vs. 0.056
CHANGE DETECTION
AUC = 0.118 vs. 0.097
F1 = 0.225 vs. 0.183
Precision = 0.135 vs. 0.102
Recall = 0.665 vs. 0.879
Minibatch Loss= 1.273800
Training: 100
ACTIVITY RECOGNITION
AUC = 0.757 vs. 0.068
F1 = 0.743 vs. 0.100
Accuracy = 0.746 vs. 0.056
CHANGE DETECTION
AUC = 0.479 vs. 0.171
F1 = 0.561 vs. 0.300
Precision = 0.440 vs. 0.176
Recall = 0.770 vs. 0.998
Validation: 100
ACTIVITY RECOGNITION
AUC = 0.418 vs. 0.064
F1 = 0.463 vs. 0.089
Accuracy = 0.796 vs. 0.056
CHANGE DETECTION
AUC = 0.356 vs. 0.094
F1 = 0.444 vs. 0.182
Precision = 0.346 vs. 0.100
Recall = 0.617 vs. 0.989
Minibatch Loss= 0.249259
Training: 200
ACTIVITY RECOGNITION
AUC = 0.903 vs. 0.066
F1 = 0.854 vs. 0.101


In [18]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../../model/WCNNRNN_Wmultitask_32_opp/WCNNRNN_Wmultitask_32_opp-10000'

In [15]:
#load the model
saver = tf.train.Saver()
saver.restore(sess, checkpoint_path+'-4495')
#saver.restore(sess, '../model/CNN7002_Gleam/CNN7002_Gleam-10000')


INFO:tensorflow:Restoring parameters from ../../model/WCNNRNN_Wmultitask_16_opp_2Adam/WCNNRNN_Wmultitask_16_opp_2Adam-4495


In [16]:
#This is the model evaluation on test dataset
_= nn_eval.evaluate_model (model, sess, test_data, test_gt)

ACTIVITY RECOGNITION
AUC = 0.603 vs. 0.060
F1 = 0.631 vs. 0.082
Accuracy = 0.911 vs. 0.056
CHANGE DETECTION
AUC = 0.337 vs. 0.053
F1 = 0.405 vs. 0.101
Precision = 0.346 vs. 0.054
Recall = 0.488 vs. 0.766


In [17]:
out = nn_eval.deploy (model, sess, test_data, test_gt)

In [20]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
#probability_change = np.squeeze(probability_of_pos, axis = 2)

In [18]:
savemat('../../poc/poc'+exp_name+'.mat', {'poc':out[:,:,-1]})

In [21]:
1

1

In [22]:
2

2